# Downloading and Converting the Kaggle MNIST-dataset

Since the Kaggle dataset contains the images of the digits in text files, I convert the data into images in this notebook, so that the conversion process does not clutter up other notebooks.

In [1]:
from fastai.vision.all import *

## Downloading the dataset from Kaggle

Let's download the dataset from Kaggle. Uncomment the next line of you do not have it on your machine yet:

In [2]:
#!kaggle competitions download -c digit-recognizer

In [3]:
#!unzip digit-recognizer.zip

As presented in the book, let's do all the file manipulation directly from the notebook:

In [4]:
path = Path('.')
Path.BASE_PATH = path

In [5]:
path.ls()

(#14) [Path('.ipynb_checkpoints'),Path('digit-recognizer.zip'),Path('mnist02-download-and-convert.ipynb'),Path('MNIST02-dummy.ipynb'),Path('MNIST02-temp.ipynb'),Path('MNIST02.ipynb'),Path('MNIST02a.ipynb'),Path('sample_submission.csv'),Path('sub1-099442.PNG'),Path('submission.csv')...]

## Inspecting the data

Let's have a look at the data so that we can convert it to actual files. For a start, let's load the csv into a dataframe:

In [6]:
import pandas as pd

train = pd.read_csv("train.csv")
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
len(train)

42000

## Displaying one image

To display one image, we can grab it from the csv-file. Essentiall, we pick a row and get all the pixel columns:

In [8]:
im = torch.tensor(train.iloc[1,1:])

In [9]:
len(im)

784

All images are 28x28 pixels:

In [10]:
math.sqrt(len(im))

28.0

Just displaying the raw data does not look nice, but if we show a fraction, we can see the shape of a number emerging:

In [11]:
im2 = torch.zeros((27,27), dtype=int)
for i in range(27):
    im2[i] = im[i*28:((i+1)*28)-1]
im2[:14,:14]

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  18,  30, 137, 137],
        [  0,   0,   0,   0,   0,   0,   0,   0,  13,  86, 250, 254, 254, 254],
        [  0,   0,   0,   0,   0,   0,   0,  16, 179, 254, 254, 254, 254, 254],
        [  0,   0,   0,   0,   0,   0,   0,  72, 254, 254, 254, 254, 254, 254],
        [  0,   0,   0,   0,   0,   0,  61, 191, 254, 254, 254, 254, 254, 109],
        [  0,   0,   0,   0,   0,   0, 172, 254, 254, 254, 202, 147, 147,  45],
        [  0,   0,   0,   0,   0,   1, 174, 254, 254,  89,  67,   0,   0,   0],
        [  0,   0,   0,   0,   0,  47, 254, 254, 254,  29,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,  80, 2

Using the elegant visualization from [chapter 4](https://github.com/fastai/fastbook/blob/master/04_mnist_basics.ipynb), here is a zero:

In [12]:
df = pd.DataFrame(im2[2:20,2:22])
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,18,30,137,137,192,86,72,1,0,0,0,0
3,0,0,0,0,0,0,13,86,250,254,254,254,254,217,246,151,32,0,0,0
4,0,0,0,0,0,16,179,254,254,254,254,254,254,254,254,254,231,54,15,0
5,0,0,0,0,0,72,254,254,254,254,254,254,254,254,254,254,254,254,104,0
6,0,0,0,0,61,191,254,254,254,254,254,109,83,199,254,254,254,254,243,85
7,0,0,0,0,172,254,254,254,202,147,147,45,0,11,29,200,254,254,254,171
8,0,0,0,1,174,254,254,89,67,0,0,0,0,0,0,128,252,254,254,212
9,0,0,0,47,254,254,254,29,0,0,0,0,0,0,0,0,83,254,254,254


Going back to the raw data, this [post](https://stackoverflow.com/questions/49330639/reading-bytes-for-an-image) nicely explains, this is how we can visualize the image:

In [13]:
image = Image.frombytes('L', (28, 28), bytes(im))
image

## Converting the csv to images

Now that we can read the csv and create an image from a line, we can simply convert the csv to image files and afterwards use the same technique as in the first notebook to classify the images (we know how to do that).

### Converting the training data

Let's create the same path structure as in Fast.AI, i.e. collect all images of a digit in one folder:

(Again, the vital calls are commented, not to accidentally create too much disk activity)

In [22]:
def prepare_paths():
    path = Path()
    path = (path/'training')

    # removing directory
    shutil.rmtree(path)

    # creating directory
    os.mkdir(path)

    for i in range(10):
        os.mkdir(path/str(i))

In [23]:
#prepare_paths()

Let's try one image:

In [24]:
def get_image(index):
    number = train.iloc[index,0]
    im = torch.tensor(train.iloc[index,1:])
    image = Image.frombytes('L', (28, 28), bytes(im))
    return number, image

In [25]:
number, image = get_image(4)
print(f'This is a {number}')
image

This is a 0


Let's save the image:

In [18]:
path = Path()
path = (path/'training')
path

Path('training')

In [19]:
image.save(path/'test.png')

All successful, let's do it for real:

In [20]:
def convert_images():
    for i in range(len(train)):
        number, image = get_image(i)
        image.save(path/str(number)/f'{i}.png')

In [21]:
#convert_images()

In [26]:
#https://codezup.com/count-files-directories-recursively-python/

noOfFiles = 0
noOfDir = 0

for base, dirs, files in os.walk(path):
    print('Looking in : ',base)
    for directories in dirs:
        noOfDir += 1
    for Files in files:
        noOfFiles += 1
        
print('Number of files',noOfFiles)
print('Number of Directories',noOfDir)
print('Total:',(noOfDir + noOfFiles))

Looking in :  training
Looking in :  training/0
Looking in :  training/1
Looking in :  training/2
Looking in :  training/3
Looking in :  training/4
Looking in :  training/5
Looking in :  training/6
Looking in :  training/7
Looking in :  training/8
Looking in :  training/9
Number of files 42002
Number of Directories 10
Total: 42012


That looks right: 42000 + 1 dummy.

### Converting the test data

The structure is slightly different, because there is no lable column:

In [29]:
test = pd.read_csv("test.csv")
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
len(test)

28000

Again, let's have a look at one image:

In [31]:
def get_test_image(index):
    im = torch.tensor(test.iloc[index])
    image = Image.frombytes('L', (28, 28), bytes(im))
    return image

In [32]:
test_image = get_test_image(0)
test_image

In [33]:
def prepare_test_path():
    path = Path()
    path = (path/'test')
    #os.mkdir(path)

    # removing directory
    shutil.rmtree(path)

    # creating directory
    os.mkdir(path)

In [34]:
#prepare_test_path()

In [35]:
path = Path()
path = (path/'test')
path

Path('test')

In [36]:
def convert_test_data():
    for i in range(len(test)):
        test_image = get_test_image(i)
        test_image.save(path/f'{i}.png')

In [37]:
#convert_test_data()

As a result, we have converted the `csv`-file to `png`-images. :)